In [1]:
import cv2 as cv
import numpy as np
import random

In [2]:
img = cv.imread("test.png")
win = cv.imread("win.png")
number_list = []
number_list.append(cv.imread("0.jpg", 0))
number_list.append(cv.imread("1.jpg", 0))
number_list.append(cv.imread("2.jpg", 0))
number_list.append(cv.imread("3.jpg", 0))
number_list.append(cv.imread("4.jpg", 0))
number_list.append(cv.imread("5.jpg", 0))
number_list.append(cv.imread("6.jpg", 0))
number_list.append(cv.imread("7.jpg", 0))
number_list.append(cv.imread("8.jpg", 0))
number_list.append(cv.imread("9.jpg", 0))
imageGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
result_ = np.copy(img)

In [1]:
import cv2 as cv
import numpy as np
import random
from functools import cmp_to_key
time_region_width = 40

class Number_container:
    def __init__(self, number, box, isColon = False):
        self.number = number
        self.box = box
        self.isColon = isColon

def find_region(method, img, template):
    w = template.shape[::-1][0]
    h = template.shape[::-1][1]
    res = cv.matchTemplate(img, template, method)
    _, _, _, max_loc = cv.minMaxLoc(res)
    return max_loc, (max_loc[0] + w, max_loc[1] + h), res

def find_boxes(method, img, template, threshold = 0.9):
    w = template.shape[::-1][0]
    h = template.shape[::-1][1]
    res = cv.matchTemplate(img, template, method)
    detected_boxes = []
    loc = np.where(res >= threshold)
    for pt in zip(*loc[::-1]):
        detected_boxes.append((pt, (pt[0] + w, pt[1] + h)))
    return detected_boxes

def box_center(box):
    return (box[0][0] + box[1][0]) * 0.5

def box_compare(item1, item2):
    if box_center(item1.box) < box_center(item2.box):
        return -1
    elif box_center(item1.box) > box_center(item2.box):
        return 1
    else:
        return 0
   

In [4]:
image = cv.imread('489.jpg')
img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
template = cv.imread('win.jpg', 0)
colon_template = cv.imread('dev.jpg', 0)
method = eval('cv.TM_CCOEFF_NORMED')

In [5]:
top_left, bottom_right, _ = find_region(method, img, template)
time_region = img[top_left[1] : bottom_right[1], bottom_right[0] : bottom_right[0] + time_region_width]

In [6]:
founded_number = []
for idx, number in enumerate(number_list):
    boxes = find_boxes(method, time_region, number)
    for box in boxes:
        founded_number.append(Number_container(idx, box))
boxes = find_boxes(method, time_region, colon_template)
for box in boxes:
   founded_number.append(Number_container(-1, box, True))
founded_number = sorted(founded_number, key=cmp_to_key(box_compare))
time = ''
for number in founded_number:
     if (number.isColon):
        time += ':'
     else:
        time += str(number.number)

In [7]:
print(time)
cv.imshow("result", time_region)
cv.imshow("img", image)
cv.waitKey(0)
cv.destroyAllWindows()

38:49


# Через сайт

In [2]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import urllib
import numpy as np
from imutils import contours
import matplotlib.pyplot as plt
from functools import cmp_to_key
import cv2 as cv


def scaleImg(image, scale: float):
    width, height = image.shape[1], image.shape[0]
    return cv.resize(image, (int(width * scale), int(height * scale)))


def clipImg(image, max_size, filter=cv.INTER_CUBIC):
    width, height = image.shape[1], image.shape[0]
    max_dim = max(width, height)
    ratio = float(max_size) / max_dim
    return cv.resize(image, (int(width*ratio), int(height*ratio)), interpolation=filter)
time_region_width = 40

def math_time(image):
    img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    # where to find match time
    template = cv.imread('win.jpg', 0)
    # seems to work 'OK'
    method = eval('cv.TM_CCOEFF_NORMED')
    top_left, bottom_right, _ = find_region(method, img, template)
    time_region = img[top_left[1]:bottom_right[1], bottom_right[0]:bottom_right[0] + time_region_width]
    # numeric templates
    numbers = []
    for i in range(10):
        numbers.append(cv.imread('' + str(i) + '.jpg', 0))
    colon_template = cv.imread('dev.jpg', 0)
    registered_numbers = []
    for idx, number in enumerate(numbers):
        boxes = find_boxes(method, time_region, number)
        for box in boxes:
            registered_numbers.append(Number_container(idx, box))
    boxes = find_boxes(method, time_region, colon_template)
    for box in boxes:
        registered_numbers.append(Number_container(-1, box, True))
    registered_numbers = sorted(registered_numbers, key=cmp_to_key(box_compare))
    time = ''
    for num in registered_numbers:
        time += ':' if num.isColon else str(num.number)
    return time, time_region, registered_numbers

def match_time_id(id, game_num=1):
    headers = {'content-type': 'application/x-www-form-urlencoded; charset=UTF-8', 'x-requested-with': 'XMLHttpRequest'}
    raw_data = 'game=&rid=matches&ajax=block_video&data%5Bid%5D=' + str(id) + '&data%5Bnum%5D=' + str(game_num)
    page = requests.post("https://game-tournaments.com/", headers=headers, data=raw_data)
    soup = BeautifulSoup(page.content, "html.parser")
    job_elements = soup.find_all("a", class_="g-rezults")
    address = job_elements[0]["href"]
    image_url = 'https://en.game-tournaments.com' + address
    page = requests.get(image_url) 
    image = np.asarray(bytearray(page.content), dtype="uint8")
    image = cv.imdecode(image, cv.IMREAD_COLOR)
    time, region, boxes = math_time(image)
    image = cv.resize(image, (1524, 520))
    return time, region, boxes, image

def match_time_number(number, game=1):
    page = np.int(number / 20) + 1
    idx = number % 20
    print(page)
    page_url = "https://game-tournaments.com/dota-2/matches"
    headers = {'content-type': 'application/x-www-form-urlencoded; charset=UTF-8', 'x-requested-with': 'XMLHttpRequest'}
    raw_data = 'game=dota-2&rid=matches&ajax=block_matches_past&data%5Bs%5D=' + str(
        page) + '&data%5Btype%5D=gg&data%5Bscore%5D=0'
    page = requests.post(page_url, headers=headers, data=raw_data)
    soup = BeautifulSoup(page.content, "html.parser")
    job_elements = soup.find_all("span", class_='mbutton tresult')
    id = job_elements[idx]["data-mid"]
    print(id)

    return match_time_id(id, game)

In [7]:
id = 25
time, region, boxes, img = match_time_number(id)
canvas = cv.cvtColor(region, cv.COLOR_GRAY2BGR)
for box in boxes:
    cv.rectangle(canvas, box.box[0], box.box[1], (200, 20, 20))
canvas = clipImg(canvas, 500, cv.INTER_NEAREST)
print(time)

C:\Users\Skiv2\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:67: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


2
418042
35:54


In [4]:
cv.imshow('time', canvas)
cv.imshow('img', img)
cv.waitKey(0)
cv.destroyAllWindows()